# Data Structures

To keep this documentation generic we typically use dimensions `x` or `y`, but this should *not* be seen as a recommendation to use these labels for anything but actual positions or offsets in space.

## Variable

### Basics

[scipp.Variable](../generated/classes/scipp.Variable.rst#scipp.Variable) is a labeled multi-dimensional array.
A variable has the following key properties:

- `values`: a multi-dimensional array of values, e.g., a [numpy.ndarray](https://numpy.org/doc/stable/reference/generated/numpy.ndarray.html#numpy.ndarray)
- `variances`: a (optional) multi-dimensional array of variances for the array values
- `dims`: a list of dimension labels (strings) for each axis of the array
- `unit`: a (optional) physical unit of the values in the array

Note that variables, unlike [DataArray](data-structures.ipynb#DataArray) and its eponym [xarray.DataArray](http://xarray.pydata.org/en/stable/user-guide/data-structures.html#dataarray), variables do *not* have coordinate dicts.

In [ ]:
import numpy as np
import scipp as sc

Variables should generally be created using one of the available [creation functions](../reference/creation-functions.rst#creation-functions).
For example, we can create a variable from a numpy array:

In [ ]:
var = sc.array(dims=['x', 'y'], values=np.random.rand(2, 4))

<div class="alert alert-info">

**Note:**

Internally scipp is not using numpy, so the above makes a *copy* of the numpy array of values into an internal buffer.
    
</div>

We can inspect the created variable as follows:

In [ ]:
sc.show(var)

In [ ]:
var

In [ ]:
var.unit

In [ ]:
var.values

In [ ]:
print(var.variances)

Variances must have the same shape as values, and units are specified using the [scipp.units](../reference/units.rst) module or with a string:

In [ ]:
var = sc.array(dims=['x', 'y'],
               unit='m/s',
               values=np.random.rand(2, 4),
               variances=np.random.rand(2, 4))
sc.show(var)

In [ ]:
var

In [ ]:
var.variances

### 0-D variables (scalars)

A 0-dimensional variable contains a single value (and an optional variance).
The most convenient way to create a scalar variable is by multiplying a value by a unit:

In [ ]:
scalar = 1.2 * sc.units.m
sc.show(scalar)
scalar

Singular versions of the `values` and `variances` properties are provided:

In [ ]:
print(scalar.value)
print(scalar.variance)

An exception is raised from the `value` and `variance` properties if the variable is not 0-dimensional.
Note that a variable with one or more dimension extent(s) of 1 contains just a single value as well, but the `value` property will nevertheless raise an exception.

Creating scalar variables with variances or with custom `dtype` or variances is possible using [scipp.scalar](../generated/functions/scipp.scalar.rst#scipp.scalar):

In [ ]:
var_0d = sc.scalar(value=1.0, variance=0.5, dtype=sc.DType.float32, unit='kg')
var_0d

In [ ]:
var_0d.value = 2.3
var_0d.variance

## DataArray

### Basics

[scipp.DataArray](../generated/classes/scipp.DataArray.rst#scipp.DataArray) is a labeled array with associated coordinates.
A data array is essentially a [Variable](../generated/classes/scipp.Variable.rst#scipp.Variable) object with attached dicts of coordinates, masks, and attributes.

A data array has the following key properties:

- `data`: the variable holding the array data.
- `coords`: a dict-like container of coordinates for the array, accessed using a string as dict key.
- `masks`: a dict-like container of masks for the array, accessed using a string as dict key.
- `attrs`: a dict-like container of "attributes" for the array, accessed using a string as dict key.

The key distinction between coordinates (added via the `coords` property) and attributes (added via the `attrs` property) is that the former are required to match ("align") in operations between data arrays whereas the latter are not.

`masks` allows for storing boolean-valued masks alongside data.
All four have items that are internally a [Variable](../generated/classes/scipp.Variable.rst#scipp.Variable), i.e., they have a physical unit and optionally variances.

In [ ]:
a = sc.DataArray(
    data = sc.array(dims=['y', 'x'], values=np.random.rand(2, 3)),
    coords={
        'y': sc.array(dims=['y'], values=np.arange(2.0), unit='m'),
        'x': sc.array(dims=['x'], values=np.arange(3.0), unit='m')},
    attrs={
        'aux': sc.array(dims=['x'], values=np.random.rand(3))})
sc.show(a)

Note how the `'aux'` attribute is essentially a secondary coordinate for the x dimension.
The dict-like `coords` and `masks` properties give access to the respective underlying variables:

In [ ]:
a.coords['x']

In [ ]:
a.attrs['aux']

Access to coords and attrs in a unified manner is possible with the `meta` property.
Essentially this allows us to ignore whether a coordinate is aligned or not:

In [ ]:
a.meta['x']

In [ ]:
a.meta['aux']

Unlike `values` when creating a variable, `data` as well as entries in the meta data dicts (`coords`, `masks`, and `attrs`) are *not* deep-copied on insertion into a data array.
To avoid unwanted sharing, call the `copy()` method.
Compare:

In [ ]:
x2 = sc.zeros(dims=['x'], shape=[3])
a.coords['x2_shared'] = x2
a.coords['x2_copied'] = x2.copy()
x2 += 123
a

Meta data can be removed in the same way as in Python dicts:

In [ ]:
del a.attrs['aux']

### Distinction between dimension coords and non-dimension coords, and coords and attrs

When the name of a coord matches its dimension, e.g., if `d.coord['x']` depends on dimension `'x'` as in the above example, we call this coord *dimension coordinate*.
Otherwise it is called *non-dimension coord*.
It is important to highlight that for practical purposes (such as matching in operations) **dimension coords and non-dimension coords are handled equivalently**.
Essentially:

- **Non-dimension coordinates are coordinates**.
- There is at most one dimension coord for each dimension, but there can be multiple non-dimension coords.
- Operations such as value-based slicing that accept an input dimension and require lookup of coordinate values will only consider dimension coordinates.

As mentioned above, the difference between coords and attrs is "alignment", i.e., only the former are compared in operations.
The concept of dimension coords is unrelated to the distinction between `coords` or `attrs`.
In particular, dimension coords could be made attrs if desired, and non-dimension coords can (and often are) "aligned" coords.

## Dataset

[scipp.Dataset](../generated/classes/scipp.Dataset.rst#scipp.Dataset) is a dict-like container of data arrays.
Individual items of a dataset ("data arrays") are accessed using a string as a dict key.

In a dataset the coordinates of the sub-arrays are enforced to be *aligned*.
That is, a dataset is not actually just a dict of data arrays.
Instead, the individual arrays share their coordinates.
It is therefore not possible to combine *arbitrary* data arrays into a dataset.
If, e.g., the extents in a certain dimension mismatch, or if coordinate values mismatch, insertion of the mismatching data array will fail.

Often a dataset is not created from individual data arrays.
Instead we may provide a dict of variables (the data of the items), and dicts for coords:

In [ ]:
d = sc.Dataset(
            data={
                'a': sc.array(dims=['y', 'x'], values=np.random.rand(2, 3)),
                'b': sc.array(dims=['y'], values=np.random.rand(2)),
                'c': sc.scalar(value=1.0)},
             coords={
                 'x': sc.array(dims=['x'], values=np.arange(3.0), unit='m'),
                 'y': sc.array(dims=['y'], values=np.arange(2.0), unit='m'),
                 'aux': sc.array(dims=['x'], values=np.random.rand(3))})
sc.show(d)

In [ ]:
d

In [ ]:
d.coords['x'].values

The name of a data item serves as a dict key.
Item access returns a new data array which is a view onto the data in the dataset and its corresponding coordinates, i.e., no deep copy is made:

In [ ]:
sc.show(d['a'])
d['a']

Use the `copy()` method to turn the view into an independent object:

In [ ]:
copy_of_a = d['a'].copy()
copy_of_a += 17  # does not change d['a']
d

Each data item is linked to its corresponding coordinates, masks, and attributes.
These are accessed using the `coords` , `masks`, and `attrs` properties.
The variable holding the data of the dataset item is accessible via the `data` property:

In [ ]:
d['a'].data

For convenience, properties of the data variable are also properties of the data item:

In [ ]:
d['a'].values

In [ ]:
d['a'].variances

In [ ]:
d['a'].unit

Coordinates of a data item include only those that are relevant to the item's dimensions, all others are hidden.
For example, when accessing `'b'`, which does not depend on the `'y'` dimension, the coord for `'y'` as well as the `'aux'` coord are not part of the item's `coords`:

In [ ]:
sc.show(d['b'])

Similarly, when accessing a 0-dimensional data item, it will have no coordinates:

In [ ]:
sc.show(d['c'])

All variables in a dataset must have consistent dimensions.
Thanks to labeled dimensions, transposed data is supported:

In [ ]:
d['d'] = sc.array(dims=['x', 'y'], values=np.random.rand(3, 2))
sc.show(d)
d

When inserting a data array or variable into a dataset ownership is shared by default.
Use the `copy()` method to avoid this if undesirable:

In [ ]:
d['a_shared'] = a
d['a_copied'] = a.copy()
a += 1000
d

The usual `dict`-like methods are available for `Dataset`:

In [ ]:
for name in d:
    print(name)

In [ ]:
'a' in d

In [ ]:
'e' in d